# Schelling's Segregation Model with Matplotlib Inline Visualisation for Jupyter Notebooks

When using Matplotlib in this form in a Jupyter notebook, it is very important to bear in mind that almost all errors generated by functions that the interactive interface call are masked/hidden, i.e. they will interupt the execution of the calling function concerned, but no error message will be visible. This can make debugging extremely difficult! It is thus crucial to find a way to avoid this during development. One possibility to achieve this is to "layer" the development, i.e. to explicitly call the step() and draw() function from the command line (Jupyter cell) first without using the interactive interface at all (This will leave error messages visible). Only in the final step, when the update functions work, should the interactive interface be wrapped around everything.  

## Set up matplotlib interface

_The next line must be commented out when the file is used via import into another notebook, but must be executed as code when this notebook is used standalone_

In [14]:
# %matplotlib nbagg

The alternative is to use the TKAgg matplotlib interface, in which case matplotlib will run in a separate window.
This can also be a bit snappier.
Note, however, that the current way the animation is implemented via a timer does not work with TkAgg. Manual stepping works fine, but auto-stepping doesn't. Messing around with timed functions in Tkagg is a bit tricky, because the Tk main loop takes over and GUI functions can only be triggered from the main thread (see https://www.physics.utoronto.ca/~phy326/python/Live_Plot.py). 

The alternative, to use Matplotlib animation, does not work either, because it cannot be interrupted. 

**Note** For development I definitely recommend toi use TKAgg, becuase nbagg will swallow all error messages, so things will simply not happen but nobody will be any wiser as to why! This could also be a problem with the students if they modify code. Maybe use nbagg for application-modulkes and TKAgg for BTS modules.

**Do not !!! execute the next line first if you want to use nbagg**

**If you want to use it you need to uncomment the code first**

In [15]:
import matplotlib
matplotlib.use('TKAgg')

## General Imports

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
import matplotlib.cm as cm

import random as RD
import scipy as SP

import time, threading

## The actual simulation code

In [17]:
def simulation(density, threshold):
    global config, fig, ax, bnext, bstart, bstop, binit, empty, unhappiness, avg_similarity
    # apparently the buttons need to be declared as global to avoid 
    # that they are garbage-collected. If this happens they will still be visible but become inactive
    speed = 1.0
    def stopAnim(d):
        global stop
        stop=True

    def startAnim(d):
        global stop
        stop=False
        foo()

    def advance(d):
        global mat, config, plt, time
        time += 1
        step(threshold)
        mat.set_data(config)
        plt.title('t = ' + str(time))    
        plt.show()

    def initAnim(d):
        global mat, config, plt, time
        time = 0
        init(density)
        mat = ax.matshow(config, cmap=cm.seismic)
        mat.set_data(config)
        plt.title('t = ' + str(time))    
        plt.show()    

    def updateSpeed(val):
        global speed
        speed = 1/sspeed.val

    def foo():
        global speed, timer, stop
        advance(None)
        if not(stop):
            timer=threading.Timer(speed, foo)
            timer.start()

    
    RD.seed()
    fig, ax = plt.subplots()
    ax.axis('off')
    plt.title("Shelling's Segregation Model")

    axspeed = plt.axes([0.175, 0.05, 0.65, 0.03])
    sspeed = Slider(axspeed, 'Speed', 0.1, 10.0, valinit=1.0)
    sspeed.on_changed(updateSpeed)

    # Call updateSpeed once to ensure speed is set
    updateSpeed(sspeed.val)

    axnext = plt.axes([0.85, 0.15, 0.1, 0.075])
    axstart = plt.axes([0.85, 0.25, 0.1, 0.075])
    axstop = plt.axes([0.85, 0.35, 0.1, 0.075])
    axinit = plt.axes([0.85, 0.45, 0.1, 0.075])
    bnext = Button(axnext, 'Next')
    bnext.on_clicked(advance)
    bstart = Button(axstart, 'Start')
    bstart.on_clicked(startAnim)
    bstop = Button(axstop, 'Stop')
    bstop.on_clicked(stopAnim)
    binit = Button(axinit, 'Init')
    binit.on_clicked(initAnim)
    config = np.zeros([height, width])
    initAnim(None)
    updateSpeed(None)

    #ani = animation.FuncAnimation(fig, advance, frames=99, interval=60, save_count=50, repeat=False)
    #plt.show()

### Define the initialisation of the state

In [18]:
def init(density):
    global config, empty, agents, unhappiness, avg_similarity
    '''
    empty: This list stores the coordinates (y, x) of all empty cells in the grid. These coordinates are used later to 
    move agents around when they are unhappy with their current location.

    agents: This list contains the coordinates (y, x) of all cells occupied by agents. Each agent's location is recorded 
    here for further processing in the model.
    unhappiness: This list is used to record the percentage of unhappy agents at each step or time point during the simulation. 
    It tracks how many agents are not satisfied with their current location.

    avg_similarity: This list tracks the average similarity of neighboring agents at each step or time point. It measures how 
    similar agents are to their neighbors.

    height and width: These are the dimensions of the grid (not defined in the function but assumed to be global variables). 
    They determine the size of the config array and the range of values for x and y.

    config: This is a 2D NumPy array with dimensions [height, width], where each cell represents a location on the grid. 
    The values in config indicate the state of each cell:
        1 indicates an agent of one type.
        -1 indicates an agent of another type.
        0 indicates an empty cell.
    '''
    empty = []
    agents = []
    unhappiness = []
    avg_similarity = []
    config = np.zeros([height, width])
    for x in range(width):
        for y in range(height):
            if RD.random() < density:
                agents.append((y, x))
                if RD.random() < 0.5:
                    config[y, x] = 1 #1: Indicates the presence of an agent of type 1.
                else:
                    config[y, x] = -1 #-1: Indicates the presence of an agent of type -1.
            else:
                config[y, x] = 0 # 0: Indicates an empty cell with no agent.
                empty.append((y, x))


### Define the state transition rules

In [19]:
def step(threshold):
    global config, agents, empty, unhappiness, avg_similarity
    '''
    threshold: A parameter that defines the minimum proportion of similar neighbors an agent needs to have to be considered "happy". 
    If an agent has fewer similar neighbors than this threshold, it is deemed unhappy and will move.

    config: This is a 2D NumPy array with dimensions [height, width], where each cell represents a location on the grid. 
    The values in config indicate the state of each cell:
        1 indicates an agent of one type.
        -1 indicates an agent of another type.
        0 indicates an empty cell.

    agents: A list of tuples representing the coordinates (y, x) of all agents on the grid.

    empty: A list of tuples representing the coordinates (y, x) of all empty cells on the grid.

    unhappiness: A list that tracks the percentage of unhappy agents at each step of the simulation.

    avg_similarity: A list that tracks the average similarity of neighboring agents at each step of the simulation.

    unhappy: A variable to count the number of unhappy agents in the current step of the simulation.

    similarity: A variable to accumulate the similarity scores of agents with their neighbors in the current step.

    height: The number of rows in the config grid, representing the vertical dimension.

    width: The number of columns in the config grid, representing the horizontal dimension.

    sequence: A shuffled list of indices for the agents. This ensures that agents are processed in a random order each time the step function is called.

    agent: The coordinates (y, x) of the current agent being processed.

    y and x: The coordinates of the current agent within the grid.

    state: The value of the cell at the agent's coordinates (y, x), which indicates the agent's type (1 or -1) or if it is empty (0).

    similar: A variable to count the number of neighbors that are similar to the current agent.

    total: A variable to count the number of non-empty neighbors around the current agent.

    dx and dy: The offsets used to check the neighboring cells relative to the current agent's position.

    v: The value of a neighboring cell, used to determine if it is occupied by an agent and if it is similar to the current agent.

    newpos: An index used to select a random empty position from the empty list.

    new_y and new_x: The coordinates of the new empty position where an unhappy agent will move.

    percent_unhappy_now: The percentage of unhappy agents at the current step, calculated as the ratio of unhappy agents to the total number of agents.

    avg_similarity_now: The average similarity score across all agents at the current step.
    '''

    unhappy = 0
    similarity = 0.0

    height, width = config.shape

    sequence = list(range(len(agents))) # A shuffled list of indices for the agents, ensuring that the order in which agents are processed is random each time.
    RD.shuffle(sequence)
    for i in sequence:
        agent = agents[i] # The current agent's coordinates.
        y, x = agent
        state = config[y, x] # either 1, -1, or 0
        if state == 0:
            continue
        similar = 0 #Counts the number of similar neighbors.
        total = 0 #Counts the total number of non-empty neighbors.
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                if not ((dx == 0) and (dy == 0)):
                    v = config[(y + dy) % height, (x + dx) % width]
                    if v != 0:
                        total += 1
                    if v == state:
                        similar += 1
        if total == 0: # If the agent has no neighbors, it's considered fully similar (hence similarity += 1).
            similarity += 1
        else: # Otherwise, the similarity is calculated as the ratio of similar neighbors to total neighbors.
            similarity += similar / (1.0 * total)
        if similar < threshold * total: # If the similarity falls below a certain threshold (indicating the agent is unhappy), the agent moves to a randomly selected empty space.
            unhappy += 1
            newpos = RD.randrange(len(empty))
            new_y, new_x = empty[newpos]
            config[new_y, new_x] = state
            agents[i] = empty[newpos]
            config[y, x] = 0
            empty[newpos] = agent
    percent_unhappy_now = unhappy / (1.0 * len(agents))  # The fraction of unhappy agents in this step (of all agents)
    avg_similarity_now = similarity / (1.0 * len(agents))  # The average similarity score across all agents.
    # These values are appended to the unhappiness and avg_similarity lists, which track the progress of the simulation over time.
    unhappiness = unhappiness + [percent_unhappy_now]
    avg_similarity = avg_similarity + [avg_similarity_now]

# Auxiliary Functions for Plotting Results

In [20]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

import numpy as np
import scipy
from scipy import stats 
from IPython.display import clear_output

In [21]:
plotly.offline.init_notebook_mode(connected=True)

In [22]:
def plot_schelling():
    global unhappiness, avg_similarity
    n = len(unhappiness)
    trace1 = go.Scatter(
        x=list(range(n)), 
        y=unhappiness, 
        mode = 'lines+markers',
        name='average unhappiness'
    )
    trace2 = go.Scatter(
        x=list(range(n)), 
        y=avg_similarity, 
        mode = 'lines+markers',
        name='average similarity'
    )
    data = [trace1, trace2]
    return iplot(data)


### Set the simulation parameters

In [23]:
width = 50
height = 50

density = 0.8  # the fraction of cells occupied (This parameter specifies the fraction of cells in the grid that will be occupied by agents. It determines how many cells will contain agents versus empty cells. A higher density means more agents are placed on the grid.)
threshold = 0.7  # the fraction of neighbors that has to be similar for an agent in order to be happy

In [24]:
unhappiness=[]
avg_similarity=[]

# Testing

**comment these lines out if the file is imported into another file**

In [25]:
simulation(density,threshold)

C:\Users\FUAD\AppData\Local\Temp\ipykernel_30940\2327171271.py:21: UserWarning:

Starting a Matplotlib GUI outside of the main thread will likely fail.



In [26]:
plot_schelling()